# file 을 나누기

In [7]:
import pandas as pd #data analysis
# import os #file/directory operations
import multiprocessing as mp
import Datacuration as dc
# from six.moves import cPickle as pickle

datafile='../../data/v2merged_test.csv' #full dataset
df=pd.read_csv(datafile)
print("\n - importing data ")
        
groups = df.groupby('ga:eventAction')
print("\n - Data has {0} individualas with {1} data".format(len(groups), len(df)))

cores = mp.cpu_count() # number of cores

dc.make_folders("../../data/pickles/multiprocessing_test/") # grouping data based on core number
print("\n - Seperate data into {0} groups including individuals".format(cores, round(len(groups)/cores)))

dc.seperate_indi(groups, cores, "../../data/pickles/multiprocessing_test/process{0}.pickle")


 - importing data 

 -Data has 4005 individualas with 13835 data

 - Seperate data into 4 groups including individuals
1001
inside
inside
inside
last!


# sleepgap.py를 실행해 result_sleepgap_15.csv 파일 생성

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Aug  2 12:03:48 2016

@author: coq15sg
"""

# -*- coding: utf-8 -*-
import numpy as np #numerical function
import pandas as pd #data analysis
import datetime 
import time
import multiprocessing as mp
from multiprocessing import Process, Queue
from six.moves import cPickle as pickle

t0 = time.clock() # initial time
cores= mp.cpu_count() #processes raw data in batches (required because data>ram)
loding = True

# parameters
long_gap_length=7*60 
gap_window=5*60 #ie long gaps are 7 hours<gap<11 hours
shor_gap_length=15 #analysis extremely robust to changes in this parameter
#guesses at what bounds waking time across the population
early_rising=5
late_rising=12
#guesses at what bounds bedtime across the population
early_daytim=early_rising+12
late_daytim=late_rising+12

def open_Pickle(pickle_path):
    try :
        with open(pickle_path, 'rb') as f:
            train = pickle.load(f)
            return train
    except Exception as e :
        print("Unable to save data to", pickle_path, ': ', e)
        return

def add_TotalPlays(data_frame):
    # add players total number of attempts to df
    data_frame['total_plays'] = len(data_frame)
    #max(data_frame['ga:eventLabel'])
    return data_frame

def add_GapType(df):
    df['localtime']=np.around((df['ga:hour']+df['ga:longitude']*24/360)%24 , decimals=1)
    df['long_gap']=(df['diff_time']>long_gap_length) & (df['diff_time']<(long_gap_length+gap_window))
    df['shor_gap']=(df['diff_time']<shor_gap_length)
    df['sleepgap']=df['long_gap'] & ((df['localtime']>early_rising) & (df['localtime']<late_rising))
    return df

def add_GapCategory(df):
    '''categorise people by their gapness'''
    '''here define range over which we define taking gaps'''
    #dfp=df[6:8] #tight range, on game 7 
    dfp=df[1:15] #any of first 15 games   
    
    if len(dfp.shor_gap)==sum(dfp.shor_gap):
        gaptype=1
    elif (sum(dfp.long_gap)==1) & (sum(dfp.sleepgap)==0):
        gaptype=2
    elif (sum(dfp.long_gap)==1) & (sum(dfp.sleepgap)==1):
        gaptype=3
    else:
        gaptype=4
        
    df['gaptype']=gaptype    
        
    return df

def add_DiffTime(df):
    for index, row in df.iterrows():
        '''
        if not index%10:
            print("{0}: {1}%".format(str(queue), round(index/len(new_df)*100)))
            
        #count = count+1
        '''
        ## combine day, hour and minute into datetime object (ie absolute time)
        str_date_time = "{0}:{1}:{2}".format(str(int(row['ga:date'])), str(int(row['ga:hour'])), str(int(row['ga:minute'])))
        str_to_time = datetime.datetime.strptime(str_date_time, "%Y%m%d:%H:%M")
        # print("{0}: {1}".format(index, aa))

        df.loc[index, 'comb_time'] = str_to_time
        
        if df.loc[index,'ga:eventLabel']==1:
            df.loc[index, 'diff_time'] = np.NaN
        else:
            try:
                diff_time = (df.loc[index,'comb_time'].to_datetime() - df.loc[index-1,'comb_time'].to_datetime()).total_seconds()/60
                df.loc[index, 'diff_time'] = diff_time
            except: #Exception as e:
                #print("Index is -1: ", e)
                df.loc[index, 'diff_time'] = np.NaN
    return df

def check_Discon(df, name_vec):    
    count = 0
    groups = df.groupby('ga:eventAction')

    for name, group in groups:
        df_len = len(group)
        eventLabel = np.ones(df_len)
        eventLabel[:] = group['ga:eventLabel']
        diff_AlOm = eventLabel[df_len-1]-eventLabel[0]+1

        if not diff_AlOm == df_len:
            #print("{0},".format(name))
            #print("{0}, length:{1}, difference:{2}".format(name, df_len, diff_AlOm))
            #print(type(name_vec))
            name_vec.append(name)
        count = count+1

def filter_time(df):
    #pre=len(df)
    df=df[~(df['ga:latitude']==0)]
    #print("latitude!=0 filter reduces players from " + str(pre) + " to " + str(len(df)) + " = " + format(round(len(df)/float(pre),2)))

    '''remove data with negative time differences'''
    '''cannot work out why some time differences are <0, but have verified that >0 values are correct'''
    #pre=len(df)
    df=df[~(df.diff_time<0)]
    #print("difftime>0 filter reduces players from " + str(pre) + " to " + str(len(df)) + " = " + format(round(len(df)/float(pre),2)))

    return df

def rm_GaInCols(data_frame): # remove ga:
    cols = np.array(data_frame.columns)
    count = 0
    for elmt in cols:
        if 'ga:' in elmt:
            cols[count] = elmt.strip('ga:')
        count = count + 1
    return cols

def process_df(queue, file_name):
    err_players = []
    
    df = open_Pickle(file_name)
    df = df.drop('Unnamed: 0', axis=1)
    
    new_df = df.groupby('ga:eventAction').apply(add_TotalPlays)
    new_df = new_df[new_df['total_plays']>14]
    new_df = new_df[new_df['ga:eventLabel']<301]
    check_Discon(new_df, err_players)
                  
    new_df = add_DiffTime(new_df)
    new_df = add_GapType(new_df)
    #df = filter_time(df) # 이건 안하는게 좋겠다. attemps를 지우니까.
    new_df=new_df.groupby('ga:eventAction').apply(add_GapCategory)
                  
    queue.put([new_df, err_players])

## Execute! ##
#print("\n - Data Accommodating")
print('\n -  Data Loading...')
# start multiprocessing
for i in range(cores):
    set_filename = "pickles/multiprocessing_test/process{0}.pickle".format(i)
    set_pros_name = "core{0}".format(i)
    set_q_name = "q{0}".format(i)

    globals()[set_q_name] = Queue()
    globals()[set_pros_name] = Process(target=process_df, args=(eval(set_q_name), set_filename,))
    #globals()[set_pros_name] = Process(target=process_df, args=(eval(set_q_name), set_filename, eval(error_name),))

    eval(set_pros_name).start()
    print('core{0} start'.format(i))
    time.sleep(1)

print('\n -  Data Loading Finish...')
## get returned value from multiprocessing
df = pd.DataFrame()
err_plys = pd.DataFrame()
for i in range(cores):
    set_q_name = "q{0}".format(i) # queue name
    set_pros_name = "core{}".format(i) # process name
    set_df_name = "df{0}".format(i) # data frame name of each process
    error_name = "error_players{0}".format(i) # player name with error attempts
    
    locals()[set_df_name], locals()[error_name] = eval(set_q_name).get() # get from queue

    df = df.append(eval(set_df_name)) # concat each dfs
    err_plys = err_plys.append(eval(error_name)) # np.hstack((err_plys, eval(error_name))) # concat each error players
    #df = df.drop('level_0', axis=1)
    
    eval(set_q_name).close()
    eval(set_pros_name).join()
    print('core{0} close'.format(i))
    time.sleep(1)

print("\n - sorting...")
df.sort_values(by=['ga:eventAction', 'ga:eventLabel'], inplace=True) # mac
#df.sort(columns=['ga:eventAction', 'ga:eventLabel'],inplace=True) # school computer
df = df.reset_index()
err_plys = err_plys.reset_index()

df = df.drop('index', axis=1)
df.columns = rm_GaInCols(df) # remove ga:

err_plys = err_plys.drop('index', axis=1)
err_plys.columns = ['player ID']

print("\n - saving processed data...\n {0} plays, {1}players".format(len(df), len(df.groupby('eventAction'))))

df.to_csv('filtered_data_test.csv')
err_plys.to_csv('error_players_test.csv')
print("\n - process terminal.")
print("Run time: ", time.clock()-t0)

# Hash table style

import pandas as pd #data analysis
import numpy as np #numerical function
from six.moves import cPickle as pickle
df=pd.read_csv('filtered_data_test.csv')
print(df.columns)
df = df.drop('Unnamed: 0', axis=1)
df = df.drop('index', axis=1)
print(df.columns)

### make a matrix that will contain each column of each player

In [ ]:
import numpy as np #numerical function
import pandas as pd #data analysis
import os #file/directory operations
from six.moves import cPickle as pickle

df=pd.read_csv('filtered_data_test.csv')
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
def spt_ColData(data_frame, col_name): # seperate columns into each matrix
    '''
    def group_Len(data, len_vec): # inner function
        len_vec.append(len(data))

    group_len = []
    groups = df.groupby('ga:eventAction') # grouping
    groups.apply(group_Len, group_len) # get length of each player

    #print(max(group_len), len(groups)) # row = attemt numbers, column = players
    '''
    groups = df.groupby('eventAction') # grouping
    col_matrix = np.ones((301, len(groups)))*np.nan # make a matrix with player attempts size.
    
    #if type(group[col_name]) == str:
    #col_matrix = np.chararray((301, len(groups)), itemsize=37)
    #col_matrix = np.zeros((301, len(groups)))

    count = 0
    for name, group in groups:
        idx = np.array(group['eventLabel'])-1
        col_matrix[idx, count] = group[col_name]
        count = count + 1
    #print('Matrix size:', np.shape(col_matrix))
    return col_matrix

def make_folders(set_dir_name):
    if os.path.exists(set_dir_name):
      # You may override by setting force=True.
      print('%s folder already present - Skipping pickling.' % set_dir_name)
    else:
        try:
            os.makedirs(set_dir_name)
        except Exception as e:
            print('Unable to make', set_dir_name, ':', e)
            return

def make_Pickle(set_data, set_filename, force = False):
    if os.path.exists(set_filename) and not force:
        # You may override by setting force=True.  
        print('%s pickle already present - Skipping pickling.' % set_filename)
    else :
        try :
            with open(set_filename, 'wb') as f:
                pickle.dump(set_data, f, pickle.HIGHEST_PROTOCOL)
        except Exception as e :
            print("Unable to save data to", set_filename, ': ', e)

In [ ]:
# print(df.columns)
mm = spt_ColData(df, 'eventLabel')
#mm = mm[:15, :]
print(mm[:, 0])

#### conver to picke except 'eventAction' and 'comb_time'

In [ ]:
def colToPickle():
    set_folder_name = 'pickles/seperate_origin/'
    make_folders(set_folder_name)    
    
    for col_name in np.array(df.columns):
        if not (col_name == 'eventAction' or col_name == 'comb_time'): # conver to picke except 'eventAction' and 'comb_time'
            set_mat = col_name
            print(col_name)
            locals()[set_mat] = spt_ColData(df, col_name)
            make_Pickle(eval(set_mat), 'pickles/seperate_origin/{0}'.format(col_name))

In [ ]:
colToPickle()

groups = df.groupby('eventAction')
names = pd.DataFrame({'ID':groups.grouper.result_index.values})
make_Pickle(names, 'pickles/seperate_origin/eventAction')